In [1]:
# prepare MEME
import pandas as pd
import numpy as np
import csv

# Build utr/orf dicts

In [2]:
filteredGeneList = pd.read_csv('../../../Database/filteredGenesDetails_human_240118.txt',sep='\t')
filteredGeneList.head(2)

,GeneName,AccNum,Chrom,mRNALength,ORFLength,fpUTR_length,tpUTR_length,sumIntronLengths
0,UBE2Q1,NM_017582,chr1,3223,1269,91,1863,6847
1,DUOXA1,NM_001276267,chr15,3032,897,395,1740,9479


In [3]:
filteredGeneList = pd.read_csv('../../../Database/filteredGenesDetails_human_240118.txt',sep='\t')
filteredGeneList.head(2)

,GeneName,AccNum,Chrom,mRNALength,ORFLength,fpUTR_length,tpUTR_length,sumIntronLengths
0,UBE2Q1,NM_017582,chr1,3223,1269,91,1863,6847
1,DUOXA1,NM_001276267,chr15,3032,897,395,1740,9479


In [4]:
acc2gene={}
for idx,row in filteredGeneList.iterrows():
    acc2gene[row['AccNum']] = row['GeneName']
acc2gene['NM_017582']

'UBE2Q1'

In [5]:
def get_fa(filename,onlyKeys = 'All'):
    keyD = dict()
    if onlyKeys!='All':
        for k in onlyKeys: keyD[k] = None
        
    def addEntry(current,d):
        entry = current.split('\n')
        if entry[0]!='' and (onlyKeys=='All' or keyD.has_key(entry[0])):
            d[entry[0]] = ''.join(entry[1:])

    d = dict()
    bf = open(filename)
    current = ['\n']
    for line in bf:
        if line[0]=='>':
            addEntry(''.join(current),d)
            if len(line)>1: current = [line[1:]]
            else: current = ['\n']
        else: current.append(line)
    addEntry(''.join(current),d)
    bf.close()
    return d

In [6]:
splicedGenes = get_fa('../../../Database/splicedGenes_240118.txt')
print(type(splicedGenes))
print(list(splicedGenes.keys())[0:3])

<class 'dict'>
['NR_148357_chr1(+):11868-14361', 'NR_046018_chr1(+):11873-14408', 'NR_024540_chr1(-):14361-29369']


In [7]:
## truncate the keys to only accNum
ks = list(splicedGenes.keys())
for key in ks:
    splicedGenes[key.split('_chr')[0]] = splicedGenes[key]
    splicedGenes.pop(key)
print(type(splicedGenes))
print(list(splicedGenes.keys())[0:3])

<class 'dict'>
['NR_148357', 'NR_046018', 'NR_024540']


In [8]:
splicedORFs = get_fa('../../../Database/splicedORFs_240118.txt')
print(type(splicedORFs))
print(list(splicedORFs.keys())[0:3])

<class 'dict'>
['NM_017582', 'NM_001276267', 'NM_173469']


In [9]:
filteredGeneList['fpUTR'] = ''
filteredGeneList['tpUTR'] = ''
filteredGeneList['ORF'] = ''
filteredGeneList.head(2)

,GeneName,AccNum,Chrom,mRNALength,ORFLength,fpUTR_length,tpUTR_length,sumIntronLengths,fpUTR,tpUTR,ORF
0,UBE2Q1,NM_017582,chr1,3223,1269,91,1863,6847,,,
1,DUOXA1,NM_001276267,chr15,3032,897,395,1740,9479,,,


In [10]:
filteredGeneList.iloc[0]['GeneName']

'UBE2Q1'

In [11]:
fpUTR={}
tpUTR = {}
ORFs = {}
for idx,row in filteredGeneList.iterrows():
    accNum = row['AccNum']
    fpLen = row['fpUTR_length']
    tpLen = row['tpUTR_length']
    fpUTR[accNum] = splicedGenes[accNum][0:fpLen]
    tpUTR[accNum] = splicedGenes[accNum][-tpLen:]
    ORFs[accNum] = splicedORFs[accNum]

In [12]:
len(filteredGeneList)

19210

In [ ]:
with open('../../../Database/fpUTR.csv','w') as f:
    for key in fpUTR.keys():
        f.write("%s,%s\n"%(key,fpUTR[key]))

In [ ]:
with open('../../../Database/tpUTR.csv','w') as f:
    for key in tpUTR.keys():
        f.write("%s,%s\n"%(key,tpUTR[key]))

In [ ]:
with open('../../../Database/ORF.csv','w') as f:
    for key in ORFs.keys():
        f.write("%s,%s\n"%(key,ORFs[key]))    

In [13]:
!pwd

/Users/zhaoy/Documents/My Projects/RBC/Deseq2 Analysis/shift15/coding


# Extract groups of interest

In [14]:
# L24 t0
L24_t0 = pd.read_csv('csv_raw/ER_L24_t0.csv')
L24_t0_filtered = L24_t0[L24_t0['padj'] < 0.05]
print(len(L24_t0_filtered))

13


#### L24 t0:
too little, ignore first

In [15]:
L24_t0_filtered = L24_t0_filtered.sort_values(by=['log2FoldChange'])
topNum=sum((L24_t0_filtered['log2FoldChange']>1)*1)
print(topNum)
botNum=sum((L24_t0_filtered['log2FoldChange']<-1)*1)
print(botNum)
L24_t0_filtered.tail()

2
0


,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
456,NM_033387,137.907415,0.776571,0.193303,4.017375,5.885001e-05,0.037928
3312,NM_001272068,164.297736,0.885366,0.205201,4.314619,1.598784e-05,0.027453
2679,NM_004074,715.242051,0.991900,0.244333,4.059630,4.915051e-05,0.035701
3598,NM_016633,9219.527608,1.510447,0.350191,4.313209,1.609022e-05,0.027453
7328,NM_001025604,1012.068880,3.970863,0.807518,4.917367,8.771593e-07,0.007009


In [16]:
L24_t1 = pd.read_csv('csv_raw/ER_L24_t1.csv')
L24_t1_filtered = L24_t1[L24_t1['padj'] < 0.05]
print('pass filter:' ,len(L24_t1_filtered))

L24_t1_filtered = L24_t1_filtered.sort_values(by=['log2FoldChange'])
topNum=sum((L24_t1_filtered['log2FoldChange']>1)*1)
print('up num: ',topNum)
botNum=sum((L24_t1_filtered['log2FoldChange']<-1)*1)
print('down num: ',botNum)
L24_t1_filtered.tail()

pass filter: 578
up num:  40
down num:  0


,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
4518,NM_019554,2782.601937,1.493255,0.381964,3.909413,9.252073e-05,4.351943e-03
2679,NM_004074,715.242051,1.584397,0.245659,6.449588,1.121548e-10,2.240292e-07
689,NM_000992,359.205401,1.593513,0.336520,4.735269,2.187652e-06,3.972577e-04
4546,NM_033625,148.300167,1.625403,0.517227,3.142532,1.674935e-03,3.020933e-02
3978,NM_005694,54.781701,1.697012,0.353176,4.805007,1.547461e-06,3.253740e-04


In [17]:
def write_fa(filename,genes,orf=False):
    if orf: #cap the total length to 600nt max
            #concate fpUTR with 100nt bases of ORF
        with open(filename,'w') as f:
            for gene in genes:
                fp = fpUTR[gene]
                if len(fp)>20:
                    fp = fp[len(fp)-20:]
                f.write(">%s\n%s\n"%(acc2gene[gene],fp+ORFs[gene][:20]))
                
    else:    
        with open(filename,'w') as f:
            for gene in genes:
                if len(fpUTR[gene])<8:
                    continue
                f.write(">%s\n%s\n"%(acc2gene[gene],fpUTR[gene]))
    

In [ ]:
write_fa('../../../MOTIF/fpUTR/L24_t1_enriched.fa',list(L24_t1_filtered.tail(topNum)['Unnamed: 0']))

In [ ]:
write_fa('../../../MOTIF/fpUTR/L24_t1_depleted.fa',list(L24_t1_filtered.head(topNum)['Unnamed: 0']))

In [ ]:
write_fa('../../../MOTIF/fpUTR/L24_t1_notEnriched.fa',list(L24_t1_filtered.head(-topNum)['Unnamed: 0']))

In [ ]:
## write for structure
write_fa('../../../MOTIF/UTRBase/L24_t1_20_enriched.fa',list(L24_t1_filtered.tail(topNum)['Unnamed: 0']),orf=True)

In [ ]:
write_fa('../../../MOTIF/UTRBase/L24_t1_depleted.fa',list(L24_t1_filtered.head(topNum)['Unnamed: 0']),orf=True)

In [ ]:
write_fa('../../../MOTIF/UTRBase/L24_t1_notEnriched.fa',list(L24_t1_filtered.head(-topNum)['Unnamed: 0']),orf=True)

## generate artificial strong MOTIF for reference

In [ ]:

with open('../../../MOTIF/fpUTR/artificial_long.fa','w') as f:
    for gene in list(L24_t1_filtered.tail(topNum)['Unnamed: 0']):
        if len(fpUTR[gene])<8:
            continue
        art_idx = int(np.random.rand()*15)
        seq = fpUTR[gene]
        if art_idx>7:
            seq = seq[0:art_idx]+'ATCCGACTGACATACGACTCT'+seq[art_idx:]
        f.write(">%s\n%s\n"%(acc2gene[gene],seq))

In [ ]:

with open('../../../MOTIF/fpUTR/fullback.fa','w') as f:
    for gene in list(L24_t0['Unnamed: 0']):
        if len(fpUTR[gene])<8:
            continue
        f.write(">%s\n%s\n"%(acc2gene[gene],fpUTR[gene]))

In [ ]:
# S15

In [18]:
S15_t1 = pd.read_csv('csv_raw/ER_S15_t1.csv')
S15_t1_filtered = S15_t1[S15_t1['padj'] < 0.05]
print('pass filter:' ,len(S15_t1_filtered))

S15_t1_filtered = S15_t1_filtered.sort_values(by=['log2FoldChange'])
topNum=sum((S15_t1_filtered['log2FoldChange']>1)*1)
print('up num: ',topNum)
botNum=sum((S15_t1_filtered['log2FoldChange']<-1)*1)
print('down num: ',botNum)
S15_t1_filtered.tail()

pass filter: 1271
up num:  189
down num:  15


,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
4546,NM_033625,148.300200,1.948511,0.515919,3.776773,1.588733e-04,2.904801e-03
5235,NM_006014,309.912400,2.084840,0.285811,7.294476,2.998229e-13,1.394083e-10
5530,NM_004965,261.013700,2.211505,0.164584,13.436970,3.671431e-41,2.933473e-37
689,NM_000992,359.205401,2.322047,0.334887,6.933830,4.095955e-12,1.422899e-09
1719,NM_000421,365.155577,3.083594,0.363617,8.480346,2.245253e-17,2.562796e-14


In [ ]:
write_fa('../../../MOTIF/fpUTR/S15_t1_enriched.fa',list(S15_t1_filtered.tail(topNum)['Unnamed: 0']))

In [ ]:
write_fa('../../../MOTIF/fpUTR/S15_t1_depleted.fa',list(S15_t1_filtered.head(topNum)['Unnamed: 0']))


In [ ]:
write_fa('../../../MOTIF/fpUTR/S15_t1_notEnriched.fa',list(S15_t1_filtered.head(-topNum)['Unnamed: 0']))
len(list(S15_t1_filtered.head(-topNum)['Unnamed: 0']))

In [ ]:
topNum

In [ ]:
## write for structure
write_fa('../../../MOTIF/UTRBase/S15_t1_enriched.fa',list(S15_t1_filtered.tail(topNum)['Unnamed: 0']),orf=True)

In [ ]:
write_fa('../../../MOTIF/UTRBase/S15_t1_depleted.fa',list(S15_t1_filtered.head(topNum)['Unnamed: 0']),orf=True)

In [ ]:
## write for structure
write_fa('../../../MOTIF/UTRBase/S15_t1_notEnriched.fa',list(S15_t1_filtered.head(-topNum)['Unnamed: 0']),orf=True)

In [19]:
norm_dat = pd.read_csv('csv_raw/deseq_normed_count.csv')
norm_dat['RPF_0'] = norm_dat[['totalRPF_0_8', 'totalRPF_0_9']].mean(axis=1)
norm_dat['RPF_1'] = norm_dat[['totalRPF_1_8', 'totalRPF_1_9']].mean(axis=1)
norm_dat['RPF_2'] = norm_dat[['totalRPF_2_8', 'totalRPF_2_9']].mean(axis=1)
norm_dat.head()

,Unnamed: 0,S24_0_8,S24_1_8,S24_2_8,S24_0_9,S24_1_9,S24_2_9,L24_0_8,L24_1_8,L24_2_8,...,RNA_2_9,totalRPF_0_8,totalRPF_1_8,totalRPF_2_8,S15_0_9,S15_1_9,S15_2_9,RPF_0,RPF_1,RPF_2
0,NM_017847,98.075215,97.527170,91.232346,82.916601,118.077866,94.707756,102.977069,78.326358,95.402696,...,187.887559,119.626839,131.960322,88.942963,93.121796,118.652650,42.843174,120.080838,136.086732,72.928319
1,NM_001003803,115.001902,153.762595,329.910908,100.220936,130.664846,369.749457,125.526792,171.471217,355.591868,...,145.185841,125.709559,193.094482,312.288627,121.774656,121.511750,236.494321,122.329206,170.009977,343.700739
2,NM_001003800,478.925670,145.897501,187.072386,603.488655,277.512953,204.983910,482.564074,143.951145,166.954718,...,307.452370,416.666362,134.196937,171.956396,667.372868,272.329276,207.360962,493.017433,202.090908,206.273009
3,NM_016649,136.409183,313.030757,182.464692,149.970896,392.593918,119.357720,145.821543,239.212932,143.104044,...,124.689017,167.274817,367.550500,175.909416,155.202993,419.572926,107.964799,196.242322,387.382632,143.574890
4,NM_016647,105.542871,124.268491,49.763098,209.094038,198.994169,171.252380,167.619608,146.068074,78.056751,...,182.763353,96.309743,77.536008,61.271819,193.406806,212.288176,157.662880,112.783748,120.434701,105.658480


In [29]:
norm_dat = norm_dat.sort_values(by=['RPF_2'])
print(len(norm_dat))
norm_dat.tail(2)

7990


,Unnamed: 0,S24_0_8,S24_1_8,S24_2_8,S24_0_9,S24_1_9,S24_2_9,L24_0_8,L24_1_8,L24_2_8,...,RNA_2_9,totalRPF_0_8,totalRPF_1_8,totalRPF_2_8,S15_0_9,S15_1_9,S15_2_9,RPF_0,RPF_1,RPF_2
922,NM_000342,94.590309,5361.241590,9.138348e+04,64.891253,2123.004059,9.718313e+04,329.225957,6115.806730,1.179741e+05,...,90384.164405,101.378677,4546.294014,9.988690e+04,101.478880,2140.036356,9.783667e+04,82.409032,3219.585715,9.445770e+04
6919,NM_000518,4215.740722,100418.772369,1.099488e+06,2384.393050,47276.099713,1.099693e+06,7097.149516,119421.234744,1.417981e+06,...,940097.110710,4261.959576,93793.221509,1.212415e+06,2616.961229,45196.652935,1.058338e+06,3133.718610,66992.957648,1.126886e+06


In [ ]:
with open('../../../MOTIF/SeqLogo/RPF2_top.seq','w') as f:
            for gene in norm_dat.tail(100)['Unnamed: 0']:
                if len(fpUTR[gene])<8:
                    continue
                fp = fpUTR[gene]
                f.write("%s\n"%(fp[len(fp)-8:]+ORFs[gene][:11]))

## Collate by percentile, output centralised seq file

In [ ]:
norm_dat = norm_dat.sort_values(by=['RPF_0'])
for i in range(0,5):
    left = len(norm_dat)//5*i
    right = len(norm_dat)//5*(i+1)
    d = norm_dat.iloc[left:right]
    filename = "../../../MOTIF/SeqLogo/RPF0_"+str(i*20)+"to"+str(i*20+20)+"percentile.seq"
    with open(filename,'w') as f:
        for gene in d['Unnamed: 0']:
            if len(fpUTR[gene])<8:
                continue
            fp = fpUTR[gene]
            f.write("%s\n"%(fp[len(fp)-8:]+ORFs[gene][:11]))

In [35]:
S15_t1 = pd.read_csv('csv_raw/ER_S15_t1.csv')
S15_t1_filtered = S15_t1[S15_t1['padj'] < 0.05]
print('pass filter:' ,len(S15_t1_filtered))

S15_t1_filtered = S15_t1_filtered.sort_values(by=['log2FoldChange'])
topNum=sum((S15_t1_filtered['log2FoldChange']>1)*1)
print('up num: ',topNum)
botNum=sum((S15_t1_filtered['log2FoldChange']<-1)*1)
print('down num: ',botNum)
S15_t1_filtered.tail()

pass filter: 1271
up num:  189
down num:  15


,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
4546,NM_033625,148.300200,1.948511,0.515919,3.776773,1.588733e-04,2.904801e-03
5235,NM_006014,309.912400,2.084840,0.285811,7.294476,2.998229e-13,1.394083e-10
5530,NM_004965,261.013700,2.211505,0.164584,13.436970,3.671431e-41,2.933473e-37
689,NM_000992,359.205401,2.322047,0.334887,6.933830,4.095955e-12,1.422899e-09
1719,NM_000421,365.155577,3.083594,0.363617,8.480346,2.245253e-17,2.562796e-14


In [28]:
with open('../../../MOTIF/SeqLogo/S15_t2_top.seq','w') as f:
            for gene in S15_t1_filtered.tail(33)['Unnamed: 0']:
                if len(fpUTR[gene])<8:
                    continue
                fp = fpUTR[gene]
                f.write("%s\n"%(fp[len(fp)-8:]+ORFs[gene][:11]))